# Experiment with Azure Digital Tween Service
[Location](https://explorer.digitaltwins.azure.net/?tid=76a2ae5a-9f00-4f6b-95ed-5d33d77c4d61&eid=digitaltwins.api.weu.digitaltwins.azure.net)

## Set up

### Create Service Instance
Create instance as described [here](https://learn.microsoft.com/en-gb/azure/digital-twins/quickstart-azure-digital-twins-explorer)

### Define models
To initial define set of models (ontology) was used Office 365 copilot and [prompt](prompts/Define%20models.txt). Some extra tuning was required.
Models are stored in `ontology` folder.
Then models were uploaded to service.

### Ontology
The service define several models as follow:

- Model name: `CentralUnit` - which is main unit of the system exists in one instance. 
    If user want to create new CentralUnit, you must refuse it with message: "Central Unit has to be created manually!"

- Model name: `ControlHub` - which is connection unit between _Central Unit_ and _Termo Sensors_. 

- Model name: `TermoSensor` - which conects to _Control Hub_ and reports temperature value.

## Gen AI
Utilise GenAI to work with toplogy. This section is designed to:
- query information
- modify state of exusted twins
- simulate various scenarios

for add/remove digital twin can be used [this notebook](01-adt-create-twins-sk.ipynb).

### Overview

![ReAct](diagrams/ReAct.png)


### Init

In [20]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the environment variables
digital_twin_endpoint = os.getenv("AZURE_DIGITAL_TWINS_ENDPOINT")

import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

Init Semantic Kernel

In [21]:
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings
from semantic_kernel.kernel import Kernel

kernel = Kernel(log=logger)

service_id = "default"
kernel.add_service(
    AzureChatCompletion(service_id=service_id),
)

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/home/vscode/.local/lib/python3.12/site-packages/certifi/cacert.pem'


### Plugin (Tools)

Define plugins that helps LLM to perform tasks on digital twins.

- `get_module_definition` - defines modules and rules of handling digital twins. It's implementation of RAG architecture where user's input _augmented_ with details of the model that needs to be maintained.
- `query_service` - requests information about _digital twin_ from Azure Digital Twins Query API.
- `update_twin` - updates properties for _digital twin_.


In [ ]:
from typing import Annotated, List
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from azure.digitaltwins.core import DigitalTwinsClient
from azure.identity import DefaultAzureCredential
import json

credentials = DefaultAzureCredential()

class AdtServicePlugin:
    """
    Description: ADTServicePlugin provides a set of functions to manipulate and query digital twins.

    Usage:
        kernel.import_plugin_from_object(AdtServicePlugin(), plugin_name="adt_servcie")

    Examples:
        {{adt_service.get_model_definition}} => Gets DTDL model's description.
    """

    client = DigitalTwinsClient(digital_twin_endpoint, credential=credentials)

    @kernel_function(
            name="get_model_definition",
            description="Returns the model definition in Digital Twin Definition Language (DTDL) format. Should be used to get model specification before operation on digital twin.")
    def get_module_definition(
        self,
        model_type: Annotated[str, "Model type that definition is required. For example: ControlHub or TermoSensor"]
    ) -> Annotated[str, "DTDL of the requested model and useful instruction how to use it.  If model name is not valid, the tool returns NA it means that execution is failed."]:
        """Gets the model definition in DTDL format by model name"""
        if model_type == "ControlHub":
            with open('ontology/ControlHub.json', 'r') as file:
                definition = file.read()

            instructions = """
            IMPORTANT NOTES:
            1. The model id in $metadata section is dtmi:com:example:ControlHub;1
            2. When requested creation new digital twin it must be performed in 2 steps:
                - create new digital twin of ControlHub model
                - create relationship between new digital twin and CentralUnit twin! ID of CentalUnit twin must be queried through Azure Digital Twins Query API. $relationship Name should be "connects"
            """
            return definition + instructions
        elif model_type == "TermoSensor":
            with open('ontology/TermoSensor.json', 'r') as file:
                definition = file.read()

            instructions = """
            IMPORTANT NOTES:
            1. The model id in $metadata section is dtmi:com:example:TermoSensor;1
            2. When requested creation new digital twin it must be performed in 2 steps:
                - create new digital twin of TermoSensor model
                - create relationship between new digital twin and ControlHub twin. 
                    $relationship name should be "connects"
                    To make this relationship you must know id of ControlHub twin, this can be obtained by several ways:
                    - user can specify id of ControlHub twin in initial request - in this case you have this information.
                    - user may specidy name of ControlHub - in this case you must query its id through Azure Digital Twins Query API.
                    - if user does not provide neither name nor id you should use available for you human tool and ask the name of ControlHub twin. And then use it to get id as specified above.
            """
            return definition + instructions
        else:
            return "NA"

    @kernel_function(
        name="update_twin",
        description="Updates one or many properties of digital twin in Azure Digital Twins Service."
    )
    def update_twin(
        self,
        twin_id: Annotated[str, "Unique identifier of the new digital twin. For example: ControlHub_1"],
        properties: Annotated[List[dict[str, object]], "Properties new values that should be updated."],
    ) -> Annotated[str, "Result of the operation. If operation is successful, the tool returns 'OK'. Otherwise, it returns 'FAIL' and the reason of failure, you're welcome to analyse error and fix it"]:
        try:
            self.client.update_digital_twin(twin_id, properties)
            return "OK"
        except Exception as e:
            print("Error creating digital twin in Azure Digital Twins service topology:", str(e))
            return f'FAIL: + {str(e)}\n'
        
    @kernel_function(
        name="query_service", 
        description="""Request information about twins or relationships in Azure Digital Twins Service.
            This function should be used whan you need to get information about particular twin or several items or relationships between them."""
        )
    def query_service(
        self,
        query: Annotated[str, """Query in Azure Digital Twins Query API format that is going to be executed on Azure Digital Twin Service.
         
        For example: You need to know the details of ControlHub twin.
        From topology you may get model definition for ControlHub iand from definition understand that id is dtmi:com:example:ControlHub;1
        Then you can format request as:
        SELECT * FROM digitaltwins T WHERE IS_OF_MODEL(T, 'dtmi:com:example:ControlHub;1')

        The request returns all description of all twins with type ControlHub."""],
    ):
        if (query == None):
            return "FAIL: Query is not defined!"

        try:
            query_result = self.client.query_twins(query)
            json_result = json.dumps(list(query_result))
            return json_result
        except Exception as e:
            print("Error deleting digital twin in Azure Digital Twins service topology:", str(e))
            return f'FAIL: + {str(e)}\n'
        
# Add plugin to the kernel
kernel.add_plugin(plugin=AdtServicePlugin(), plugin_name="adt_servcie")

### Prompt engineering

Define _system prompt_ that setup the stage.

In [6]:
system_message = """
    I am a user who is working on support digital twins in Azure Digital Twins Service. I will need you support to perform operation that I will request.
    
    The service defines several models (ontology) as follow:
    - Model CentralUnit - which is main unit of the system exists in one instance. 
        You cannot create CentralUnit as it is already defined. If the creation will be requested refuse it.
    - Model ControlHub - which is connection unit between CentralUnit and TermoSensors.
    - Model TermoSensor - which conects to ControlHub and reports temperature value.

    My requests will be related to change some properties of existed twins. You should identify types of twins that I request to update and also relationships between them.

    Relationship between digital twins are stored separately from twins. If user request information where connection between twins - you must request
    relationship information from relationships collection!
    For example for question: what temperature sensor connected to hub with id Node_23 valid query will be:
    SELECT * FROM relationships WHERE $sourceId = 'Node_23'
    Note request was done for relationships not digitaltwins collection.

    Relationship collection contains following information about relationship (it is example):
    {
        "$relationshipId": "ControlHub_Dnepr_TermoSensor_Dnepr_1",
        "$sourceId": "ControlHub_Dnepr",
        "$targetId": "TermoSensor_Dnepr_1",
        "$relationshipName": "connects",
        "$etag": "W/\"1dd250e7-6b1a-4fc4-bdbe-31c10583bd6d\""
    },
    ignore $etag field as it is not important for you.

    If changes of properties for digital twin is required, you must use proper tool. Please be very careful with parameters that should be
    provided to the tool. 1st parameter is digital twin id and 2nd parameter is list of properties that should be updated.
    list of properties is a list of objects (dictionaries), format of each property is following:
        {
            "op":"add",
            "path":"/property_name",
            "value":property_value
        }
        where:
            "op" - operation wih value "add", nothing else is accepted by function!
            "path" - path to the property (property name) that should be updated, see example bellow
            "value" - value that should be set.
                              
        For example: to change the property "temperature" to 35 and tge property "status" to "on" for TermoSensor twin, the properties parameter should be:
            propertiess = [
            {
                "op": "add",
                "path": "/status",
                "value": "on"
            },
            {
                "op": "add",
                "path": "/temperature",
                "value": 35
            }
        ]                            

    For requests you must analyse properties of each twin and make appropriate queries to digitaltwins collection to get information about twins.
    Example 1: if you requested all twins located in Kyiv, you must request all twins of appropriate type that are located in Kyiv. For this operation 
    relationship is not required.
    Example 2: if you requested for all temperature sensors connected to any hub located in Odesa - you must request all ControlHub with location = Odesa
    from digitaltwins collection and then request relationships collection to get all relationships where $sourceId is ControlHub twin id.

    The request is following:
    
    """


### Query

Is used ReAct approach that let LLM to think what steps should be done to achieve task and what tool is better fit this needs.
Several agent were cinsidered from [available](https://github.com/microsoft/semantic-kernel/blob/main/python/samples/getting_started/05-using-the-planner.ipynb)

Set up the planner

In [ ]:
from semantic_kernel.planners.function_calling_stepwise_planner import (
    FunctionCallingStepwisePlanner,
    FunctionCallingStepwisePlannerOptions,
)

options = FunctionCallingStepwisePlannerOptions(
    max_iterations=10,
    max_tokens=4000
)

planner = FunctionCallingStepwisePlanner(service_id=service_id, options=options)

#### Common query

In [9]:
request = "How many TermoSensor do we have in out topology?"
request = "How many TermoSensors connected to Hub_3?"

#### Predictable Maintenance
forecast demand, topology analysys, simulation traffic

In [7]:
request="Please create Control Hub with following parameters: name: Hub_SK, Location: Kernel, Status: on."

- Analytics

In [8]:
request="How many termosensors does not have temperature value? I need also names of these sensors." # GOOD!

In [14]:
request="please find top 3 hotter termosensors" # GOOD!

In [17]:
request="I'm lloking for location with temperature anomaly. Can you find location that I'm lloking for? By anomaly I assume temperature more than 35"

! Interesting: for the request 
```
request="I'm looking for any units that status is not on. Please provide me list of Id such units."
```

LLM is not able to find right tools and provide instructions to user....

```
To address your request, we need to identify all units (ControlHub and TermoSensor) whose status is not &quot;on&quot;. Here is the step-by-step plan to achieve this:

1. Query the digitaltwins collection to get all ControlHub twins.
2. Query the digitaltwins collection to get all TermoSensor twins.
3. Filter the results from both queries to find twins where the status property is not &quot;on&quot;.
4. Collect the IDs of these twins and provide the list.

Since we don't have the actual functions to perform these queries and filter operations, I will provide the final answer based on the logical steps described.
```

In [44]:
request="""I'm looking for any units that status is not on.
Please provide me list of Id such units. 
Once you know the query you should query the service with tools available for you."""

- direct control

In [11]:
request="Please set temperature to 18 for termosensor TermoSensor_Dnepr_1"

#### Batch Operations

Request to add/remove to topology several twins as one user's request

In [41]:
request="""For termosesnsors located in Odesa please set temperature 42 and status to on""" # GOOD

### Execution

Debug tips:
 - function call starts with `INFO:semantic_kernel.kernel:Calling`

In [ ]:
question = system_message + request

result = await planner.invoke(kernel, question)

In [46]:
print(result.final_answer)

Here is the list of units where the status is not "on":
- TermoSensor_Dnepr_3
- TermoSensor_Lviv_1
- TermoSensor_Lviv_2
- TermoSensor_Lviv_3
- TermoSensor_Lviv_4
